In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/image_classification_course

In [ ]:
!pwd

**Steps to Train an Image Classfiction model**


1.   Loading the train and test dataset
2.   Loading the model
3.   Hyper-Parameters
4.   Training Loop
5.   Check the model accuracy
6.   Saving the model checkpoint or weight



**Import Package**

In [ ]:
import os
from time import sleep
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report
from tqdm import tqdm

from dataset import load_data
from models.lenet import LeNet

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
print(device)


**Loading the train and test dataset**

In [ ]:
dataset_path = 'data/version1'
image_size = (32,32)
batch_size = 16

train_data, test_data = load_data(dataset_path, image_size, batch_size)

**Loading the model**

In [ ]:
in_channel = 3
num_classes = 7

model = LeNet(in_channel= in_channel, num_classes= num_classes)

model = torch.compile(model)

model.to(device)

model.train()

**Hyper parameters loss and optimizer**

In [ ]:
learning_rate = 1e-3
num_epochs = 30

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr = learning_rate)

**Model accuracy check**

In [ ]:
def get_model_accuracy(test_data, model):
    # Initialize the prediction and label list
    y_pred = np.zeros(0)
    y_true = np.zeros(0)

    model.eval()
    with torch.no_grad():
        for images_batch, y_true_batch in test_data:
            images_batch = images_batch.to(device)

            scores = model(images_batch)
            _, y_pred_batch = scores.max(1)
            y_pred_batch = y_pred_batch.cpu()

            y_pred = np.concatenate((y_pred, y_pred_batch))
            y_true = np.concatenate((y_true, y_true_batch))

    model.train()

    report = classification_report(y_true, y_pred)
    print(report)

**Training loop**

In [ ]:
for epoch in range(1, num_epochs + 1):
    pbar_batch = tqdm(train_data, unit="batch")
    losses = []
    for data in pbar_batch:
        pbar_batch.set_description(f"Epoch {epoch}")
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        scores = model(images)
        loss = criterion(scores, labels)
        losses.append(loss.item())
        cost = sum(losses)/len(losses)

        # backward

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pbar_batch.set_postfix(loss=cost)
        sleep(0.1)

    if (epoch % 5) == 0:
        get_model_accuracy(test_data, model)
        sleep(0.1)


print("model training complete")

**Saving the model checkpoint**

In [ ]:
model_save_path = "weights/lenet"
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

torch.save(model.state_dict(), model_save_path + "/lenet_model_checkpoint.pth")